
# Step-by-step agent state management


In this section, you will see how the agent state is managed in our framework.

## The big picture


The following diagram gives you a high-level understanding of how the agent state management works for a generic negotiation.

.. mermaid:: ../diagrams/agent-state-management-guide/summary.mmd

Let's see step by step what happens:

1. `Agent_1` sends a `CFP` to `Agent_2`, meaning that she wants to start a negotiation. The CFP contains a reference to the goods which `Agent_1` is interested in and whether `Agent_1` is a buyer or seller  of these goods.

2. Assuming `Agent_2` does not decline the CFP, `Agent_2` calls `get_proposals()` to generate a list of proposal for answering to `Agent_1`. When generating the proposals `Agent_2` applies all locked transactions - we discuss below when a transaction becomes locked - to the current agent state to generate a "forward looking agent state" (i.e. the state the agent will be in when all locked transactions have been settled by the `Controller`). This ensures `Agent_2` takes into account all the transactions she has committed to.
3. `Agent_2` calls `store_proposals()` to store the proposals as `pending_proposals`.
4. `Agent_2` replies with a `Propose` message which includes the proposals as an answer to the `CFP` (note, currently the list of proposals includes exactly one proposal).

5. `Agent_1` translates the proposal into a transaction and calls `is_profitable_transaction()` to identify whether the transaction is profitable.
6. Assuming the proposal is profitable, `Agent_1` calls `add_locked_tx()` to add the transaction to the list of locked transactions.
7. `Agent_1` calls `add_pending_initial_acceptance()` to add the transaction to the list of pending initial acceptances. This helps the agent identify whether an incoming `Accept` from `Agent_2` is a matching accept or an initial accept.
8. `Agent_1` sends an `Accept` message to `Agent_2`, meaning that she accepts the proposal.

9. `Agent_2` calls `pop_pending_proposal()` to recover the proposal -  in the form of a transaction - made to `Agent_1` and referenced in the acceptance message.
10. `Agent_2` calls `is_profitable_transaction()` to identify whether the transaction is profitable. This proposal was created by `Agent_2`, however in the meanwhile `Agent_2` might have a different agent state which could render this proposal no longer profitable.
11. Assuming the proposal is still profitable, `Agent_2` calls `add_locked_tx()` to add the transaction to the list of locked transactions.
12. `Agent_2` sends an `Accept` message to `Agent_1`, meaning that she "match-accepts" the proposal.
13. `Agent_2` sends a Transaction request to the `Controller`.

14. `Agent_1` calls `pop_pending_initial_acceptance()` to 
15. `Agent_1` sends a Transaction request to the `Controller` (analogous to step 13).

16. The `Controller` notifies `Agent_1` that the transaction has been confirmed.
17. `Agent_1` calls `pop_locked_tx()` to remove the transaction from the locked transaction list.
18. `Agent_1` calls `agent_state.update()` to update its state.

19. The `Controller` notifies `Agent_2` that the transaction has been confirmed.
20. `Agent_2` calls `pop_locked_tx()` to remove the transaction from the locked transaction list (analogous to step 17).
21. `Agent_2` calls `agent_state.update()` to update its state (analogous to step 18).